# Source laoding, basic setup

In [7]:
!conda info --envs

# conda environments:
#
base                  *  /mmfs1/apps/easybuild/software/Miniforge3/24.1.2-0
                         /mmfs1/project/sa2564/dv336/fine/llm



In [1]:
!pwd

/project/sa2564/dv336/fine


In [8]:
!nvidia-smi

Fri Sep 20 00:51:18 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:41:00.0 Off |                    0 |
| N/A   26C    P0             60W /  500W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

# Begin

In [1]:
!nvidia-smi

Fri Sep 20 10:17:59 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   28C    P0             60W /  500W |       0MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
import torch
import torch.nn as nn
from accelerate import Accelerator
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM, BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
   bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


base_model = AutoModelForCausalLM.from_pretrained(
    "bigcode/starcoder2-7b",
    quantization_config=bnb_config,
    use_cache=False,
    device_map="auto"
)

config.json:   0%|          | 0.00/893 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/41.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.51G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# Model load

In [3]:
model_id = "bigcode/starcoder2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [4]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [5]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

base_model = prepare_model_for_kbit_training(base_model)
model = get_peft_model(base_model, lora_config)
print_trainable_parameters(model)

trainable params: 39845888 || all params: 3740886016 || trainable%: 1.0651457389927594


In [6]:
from datasets import load_dataset, Dataset
train_data = load_dataset("json", data_files="beta_train_all.json")
#test_data = load_dataset("json", data_files=$testdatasetpath$)

In [7]:
print(train_data)

DatasetDict({
    train: Dataset({
        features: ['description', 'metrics', 'code'],
        num_rows: 4000
    })
})


In [12]:
def tokenize(prompt, add_eos_token=True):
    # Tokenize the prompt
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=3000,
        padding=False,
        return_tensors=None,
    )
    # Add EOS token if needed
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < 3000
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    # Set labels to be the same as input_ids for language modeling
    result["labels"] = result["input_ids"].copy()

    return result

def generate_prompt(example) -> str:
    # Manually construct the conversation-like prompt
    system_prompt = "You are a helpful assistant. Please generate the Verilog header by taking the needed description:"
    user_prompt = example['description']

    # Create the full prompt by concatenating the system and user messages
    full_prompt = f"<s>[SYSTEM] {system_prompt}</s>\n<s>[USER] {user_prompt}</s>\n"
    
    # Append the actual code if needed
    if 'code' in example:
        full_prompt += f"<s>[CODE] {str(example['code'])}</s>\n"

    # Tokenize the full prompt
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt


In [13]:
train_val = train_data["train"].train_test_split(
    test_size=500, shuffle=True, seed=42
)
train_data = (
    train_val["train"].shuffle().map(generate_prompt)
)
val_data = (
    train_val["test"].shuffle().map(generate_prompt)
)

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [15]:
print(val_data)

Dataset({
    features: ['description', 'metrics', 'code', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 500
})


In [16]:
train_data = train_data.remove_columns(train_data['train'].column_names)
val_data = val_data.remove_columns(train_data['train'].column_names)

KeyError: "Column train not in the dataset. Current columns in the dataset: ['description', 'metrics', 'code', 'input_ids', 'attention_mask', 'labels']"

In [17]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./ftstar",
    num_train_epochs=3, # replace this, depending on your dataset
    per_device_train_batch_size=1,
    learning_rate=1e-5,
    optim="adamw_torch",
    gradient_accumulation_steps=16,
    logging_steps=10,
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps= 300,
    save_steps=300,
    save_total_limit=4,
    load_best_model_at_end=True
)

In [18]:
import torch
# Step 1: Import AdamW from torch.optim
optimizer = torch.optim.AdamW(model.parameters(), lr=args.learning_rate)

# Step 2: Initialize the accelerator
accelerator = Accelerator()

# Step 3: Prepare the model, optimizer, and dataloaders for multi-GPU or distributed training
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_data, val_data
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [ ]:
import transformers
trainer = transformers.Trainer(
    model=model,
    tokenizer=tokenizer,
    #lora_config= lora_config,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=args,
    data_collator=transformers.DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/mmfs1/project/sa2564/dv336/fine/llm_finetuning_venv/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/mmfs1/project/sa2564/dv336/fine/llm_finetuning_venv/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.a

In [ ]:
from transformers import Trainer

trainer.save_model("./ft_starcoder_7b")

# Load the model and test the output

In [ ]:
eval_results=trainer.evaluate()

In [ ]:
print(f"Hogyaya..!! Results :{eval_results}")